In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [ ]:
train_df = pd.read_csv("../input/train.csv",nrows = 200000)

In [ ]:
train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime'])

In [ ]:
train_df.describe()

One of the major no no from this statistical analysis is there are many outliers that may lead us to wrong predictions.

Mean of fare amount is 11.35 where as the minimum is -44 and maximum is 200. We can definitely conclude that there are many definite outliers here.

Same is true for almost all the columns of our datframe hence it would be wise to remove such outliers before modelling. But first let us visualize out data so that the situation of such outliers will be more clear.


In [ ]:
plt.figure(figsize = (10,6))
sns.distplot(train_df['fare_amount']);
plt.title('Distribution of fare')

As we can see most of the fare prices lies between 0-50 rest are all outliers.

In [ ]:
#removing values below 0. Certainly an wrong entry.. may be cabbi has broken some rule and hence fined.
#whatever it is we need to remove them.
train_df[(train_df['fare_amount']<0)].shape[0]#13 such entries

In [ ]:

train_df = train_df[(train_df['fare_amount']>=0)]
train_df = train_df[(train_df['fare_amount']<=100)]
train_df['fare_amount'].size 

In [ ]:
plt.figure(figsize = (10,6))
sns.distplot(train_df['pickup_latitude']);
plt.title('Distribution of latitude')


In [ ]:
#40.7128° N  <-- New York City latitude
train_df['pickup_latitude'].mean() #mean of the data we have. (kinda accurate) Fact: 1 degree latitude is 69 miles so all the values other than 38-42 are outliers

In [ ]:
plt.figure(figsize = (9,4))
sns.distplot(train_df['pickup_latitude'])
plt.xlim(0,80)
plt.title('Distribution of latitude in more detail')
#There is lot of values between 10-20. I dont know why

In [ ]:
cnt = []
r = [35,36,37,38,39,40,41,42,43,44,45]
for i in r:
    no = train_df[(train_df['pickup_latitude']<i)].shape[0]
    cnt.append(no)
plt.scatter(r,cnt) #see the jump

In [ ]:
train_df = train_df[(train_df['pickup_latitude']>0)]
train_df = train_df[(train_df['pickup_latitude']>39)] #-->111 #there are only 20 <39 and 111 greater than 4 so removing them
train_df = train_df[(train_df['pickup_latitude']<42)]
#we can see mean has increased
train_df['pickup_latitude'].mean() 

In [ ]:
#longitude
#NYC longitude -> 74.0060° W
train_df['pickup_longitude'].mean() #-->-73.96832831445111
plt.figure(figsize = (10,6))
sns.distplot(train_df['pickup_longitude']);
plt.title('Distribution of longitude') #There are outliers nee to remove them

In [ ]:
train_df = train_df[(train_df['pickup_longitude']>-75)]
train_df = train_df[(train_df['pickup_longitude']<-72)]

In [ ]:
train_df['dropoff_longitude'].mean() #-->-73.90602772217527
train_df['dropoff_latitude'].mean() #--> 40.72183614573583
train_df.shape[0]

In [ ]:
train_df = train_df[(train_df['pickup_longitude']>-75)]
train_df = train_df[(train_df['pickup_longitude']<-72)]
train_df['dropoff_longitude'].mean() #->-73.90602772217527

In [ ]:
train_df = train_df[(train_df['pickup_latitude']>0)]
train_df = train_df[(train_df['pickup_latitude']>39)] #-->111 #there are only 20 <39 and 111 greater than 4 so removing them
train_df = train_df[(train_df['pickup_latitude']<42)]
train_df['dropoff_latitude'].mean()  #->40.72183614573583

Now last feature no of passengers

In [ ]:
train_df['passenger_count'].describe()

In [ ]:

plt.figure(figsize = (10,6))
sns.distplot(train_df['passenger_count']);
plt.title('passenger_count') #At this moment I dont think there is any need to preprocess this data

**So preprocessing is done lets take overview**

In [ ]:
train_df.describe()

**Data is preprocessed. I think generally fare amount will depend on the distance travelled by the vehicle. So we need to calculate the distance travelled by the vehicle. Another important point is New York has streets in grid so we may need to find the distance
Now Important point is lattitude and longitude are not cartessian co ordinates so can't use simple distance formula insted have to use haversine formula[1]. Which is: {\displaystyle \operatorname {hav} \left(\Theta \right)=\operatorname {hav} (\varphi _{2}-\varphi _{1})+\cos(\varphi _{1})\cos(\varphi _{2})\operatorname {hav} (\lambda _{2}-\lambda _{1})} <--- This is Scary so I am going to use ready made github code with changing math library to numpy[2]
[1] https://math.stackexchange.com/questions/29157/how-do-i-convert-the-distance-between-two-lat-long-points-into-feet-meters
[2] https://gist.github.com/rochacbruno/2883505
****

In [ ]:
def distance(pickup1,pickup2,destination1,destination2):
    lat1, lon1 = pickup1,pickup2
    lat2, lon2 = destination1,destination2
    radius = 3959 # miles

    dlat = np.radians(lat2-lat1)
    dlon = np.radians(lon2-lon1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c

    return d

In [ ]:
a = train_df['pickup_latitude'].values
b = train_df['pickup_longitude'].values
c = train_df['dropoff_latitude'].values
d = train_df['dropoff_longitude'].values

In [ ]:
dist = distance(a,b,c,d)

In [ ]:
dist[0:20]

In [ ]:

dist = pd.Series(dist)


In [ ]:
train_df['distance'] = dist

In [ ]:
train_df.describe()

In [ ]:
train_df[train_df['distance']>500]

In [ ]:
dist3 = distance(40.762892,-73.976545,40.76406,-73.79317)
dist3

In [ ]:
plt.figure(figsize = (10,6))
sns.distplot(train_df['distance']>100);
plt.title('passenger_count') 

In [ ]:
def distance1(pickup1,pickup2,destination1,destination2):
    a = (destination1 - pickup1)*69
    b = (destination2 - pickup2)*69
    d = a**2+b**2
    c = np.sqrt(d)
    return c

In [ ]:
dist1 = distance1(a,b,c,d)

In [ ]:
dist1 = pd.Series(dist1)

In [ ]:
dist1.max()

**Pickup laitude**

In [ ]:
plt.figure(figsize = (10,6))
sns.kdeplot(train_df['pickup_latitude'])
sns.kdeplot(train_df['pickup_longitude']);
plt.title('Distribution of pickup lat and long')
plt.figure(figsize = (10,6))
sns.kdeplot(train_df['dropoff_latitude']);
sns.kdeplot(train_df['dropoff_longitude']);
plt.title('Distribution of dropoff lat and long')